In [1]:
%reload_ext autoreload
%autoreload 2

In [10]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex

data: DiffResult = load_pkl("src")


col_names = {
    IssueEnum.OLD: r"\multicolumn{1}{c}{\ No.}",
    IssueEnum.ADD: r"\multicolumn{1}{c}{$+$\tiny\%}",
    IssueEnum.REMOVE: r"\multicolumn{1}{c}{$-$\tiny\%}",
    IssueEnum.CHANGE: r"\multicolumn{1}{c|}{$\Delta$\tiny\%}",
}

group_names = {
    VersionGroup.LTS: "LTS (2 yr)",
    VersionGroup.REGULAR: "Regular Releases (6 mo)",
    # VersionGroup.REV: "Revisions for 5.4",
}


results = {}

for (group, pair), diffs in data.items():
    if group not in group_names:
        continue

    result = {}
    old = 0
    for (kind, key), s in diffs.items():
        if kind == DepKind.SYSCALL:
            continue

        if key not in col_names.keys():
            continue

        display_key = col_names[key]

        if key == "Old":
            old = s
            if kind == DepKind.FUNC:
                s = f"{s / 1000:.0f}k"
            elif kind == DepKind.STRUCT:
                s = f"{s / 1000:.1f}k"
        else:
            if s == 0:
                s = GRAY_DASH
            else:
                s = f"{s / old * 100:.0f}"

        s = r"\multirow{2}{*}{" + str(s) + "}"
        result[(kind, display_key)] = s

    v1 = group.version_to_str(pair.v1)
    v2 = group.version_to_str(pair.v2)

    group_name = group_names[group]
    results[(group_name, v1)] = result
    if pair.v2 == list(group)[-1]:
        results[(group_name, v2)] = {k: "" for k in result.keys()}


df = pd.DataFrame(results).T

latex = df.to_latex(multicolumn_format="c|", column_format="cc|rrrr|rrrr|rrrr|rrrr")
save_latex(latex, "src")
df

[          pkl.py:18 ] INFO: Loding src from /Users/szhong/Downloads/bpf-study/output/src.pkl
[        latex.py:48 ] INFO: Saved src to /Users/szhong/Downloads/bpf-study/paper/tabs/src.tex


Function                                                                                                                       Struct                                                                                                                   Tracepoint                                                                                                                             LSM Hook                                                                                                                     
                             \multicolumn{1}{c}{\ No.} \multicolumn{1}{c}{$+$\tiny\%} \multicolumn{1}{c}{$-$\tiny\%} \multicolumn{1}{c|}{$\Delta$\tiny\%} \multicolumn{1}{c}{\ No.} \multicolumn{1}{c}{$+$\tiny\%} \multicolumn{1}{c}{$-$\tiny\%} \multicolumn{1}{c|}{$\Delta$\tiny\%} \multicolumn{1}{c}{\ No.} \multicolumn{1}{c}{$+$\tiny\%}         \multicolumn{1}{c}{$-$\tiny\%} \multicolumn{1}{c|}{$\Delta$\tiny\%} \multicolumn{1}{c}{\ No.}         \multicolumn{1}{c}{$+$\tiny\%}         \multicolumn{1}{c}{$-$\tiny\%}   \multicolumn{1}{c|}{$\Delta$\tiny\%}
LTS (2 yr)              4.4       \multirow{2}{*}{36k}            \multirow{2}{*}{24}             \multirow{2}{*}{7}                   \multirow{2}{*}{5}     \multirow{2}{*}{6.2k}            \multirow{2}{*}{24}             \multirow{2}{*}{4}                  \multirow{2}{*}{18}      \multirow{2}{*}{502}            \multirow{2}{*}{39}                     \multirow{2}{*}{5}                   \multirow{2}{*}{8}      \multirow{2}{*}{168}                     \multirow{2}{*}{9}                     \multirow{2}{*}{4}                    \multirow{2}{*}{11}
                        4.15      \multirow{2}{*}{42k}            \multirow{2}{*}{21}             \multirow{2}{*}{8}                   \multirow{2}{*}{4}     \multirow{2}{*}{7.3k}            \multirow{2}{*}{19}             \multirow{2}{*}{4}                  \multirow{2}{*}{15}      \multirow{2}{*}{675}            \multirow{2}{*}{15}                     \multirow{2}{*}{3}                   \multirow{2}{*}{8}      \multirow{2}{*}{177}                     \multirow{2}{*}{8}                     \multirow{2}{*}{1}                    \multirow{2}{*}{10}
                        5.4       \multirow{2}{*}{48k}            \multirow{2}{*}{22}            \multirow{2}{*}{10}                   \multirow{2}{*}{5}     \multirow{2}{*}{8.4k}            \multirow{2}{*}{17}             \multirow{2}{*}{6}                  \multirow{2}{*}{16}      \multirow{2}{*}{752}            \multirow{2}{*}{14}                     \multirow{2}{*}{5}                  \multirow{2}{*}{16}      \multirow{2}{*}{189}                     \multirow{2}{*}{8}                     \multirow{2}{*}{1}                    \multirow{2}{*}{14}
                        5.15      \multirow{2}{*}{54k}            \multirow{2}{*}{23}             \multirow{2}{*}{8}                   \multirow{2}{*}{6}     \multirow{2}{*}{9.3k}            \multirow{2}{*}{16}             \multirow{2}{*}{4}                  \multirow{2}{*}{16}      \multirow{2}{*}{818}            \multirow{2}{*}{18}                     \multirow{2}{*}{4}                  \multirow{2}{*}{14}      \multirow{2}{*}{203}                    \multirow{2}{*}{10}                     \multirow{2}{*}{2}                    \multirow{2}{*}{13}
                        6.8                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
Regular Releases (6 mo) 4.4       \multirow{2}{*}{36k}             \multirow{2}{*}{8}             \multirow{2}{*}{3}                   \multirow{2}{*}{2}     \mu

In [3]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex, use_midrule

data: DiffResult = load_pkl("src")


def multicolumn(s, n=2, format="c"):
    return r"\multicolumn{2}{ " + format + r"}{" + s + "}"


KINDS = {
    DepKind.FUNC: DepKind.FUNC,
    DepKind.STRUCT: DepKind.STRUCT,
    DepKind.TRACEPOINT: "Tracept",
}

results = {}
for (group, pair), diffs in data.items():
    if group != VersionGroup.LTS:
        continue

    result = {}
    change = 0
    for (kind, key), v in diffs.items():
        if v == 0:
            continue
        if key in (IssueEnum.ADD, IssueEnum.REMOVE, IssueEnum.OLD, IssueEnum.NEW):
            continue
        if kind == DepKind.TRACEPOINT and key not in (
            IssueEnum.CHANGE,
            IssueEnum.TRACE_EVENT_CHANGE,
            IssueEnum.TRACE_FUNC_CHANGE,
        ):
            continue
        kind = KINDS.get(kind)
        if kind is None:
            continue

        if key == IssueEnum.CHANGE:
            change = v
            key = "No. changed"
            v = f"{v / 1000:.1f}k" if v > 1000 else str(v)
        else:
            key = f" - {key}"
            v = f"{v / change * 100:.0f}" + r"\%"
        
        result[(kind, key)] = v

    v1 = group.version_to_str(pair.v1)
    v2 = group.version_to_str(pair.v2)

    def header(v):
        return ("Linux Kernel Version", v)

    if pair.v1 == list(group)[0]:
        results[header(v1)] = {k: "" for k in result.keys()}
    results[header(v2)] = result

df = pd.DataFrame(results)
df = df.fillna(GRAY_DASH)
df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: x + " &")
latex = df.to_latex(multicolumn_format="c", column_format="cl|" + "r" * 7)
latex = latex.replace(
    r"\multicolumn{5}{c}{Linux Kernel Version}",
    r"\multicolumn{6}{c}{Linux Kernel Version}",
)
latex = latex.replace(
    "4.4 & 4.15 & 5.4 & 5.15 & 6.8",
    r"\multicolumn{6}{c}{4.4 \quad 4.15 \quad 5.4 \quad 5.15 \quad 6.8}",
)
latex = use_midrule(latex)
save_latex(latex, "breakdown")
df

[          pkl.py:18 ] INFO: Loding src from /Users/szhong/Downloads/bpf-study/output/src.pkl
[        latex.py:48 ] INFO: Saved breakdown to /Users/szhong/Downloads/bpf-study/paper/tabs/breakdown.tex


Linux Kernel Version                          
                                                 4.4  4.15   5.4  5.15     6.8
Function No. changed                                  2.0k  1.8k  2.2k  3.0k &
          - Param added                               57\%  53\%  60\%  51\% &
          - Param removed                             41\%  36\%  44\%  48\% &
          - Param reordered                           19\%  19\%  25\%  19\% &
          - Param type changed                        26\%  23\%  26\%  25\% &
          - Return type changed                       15\%  21\%  13\%  17\% &
Struct   No. changed                                  1.1k  1.1k  1.3k  1.5k &
          - Field added                               72\%  74\%  75\%  74\% &
          - Field removed                             41\%  40\%  40\%  42\% &
          - Field type changed                        37\%  34\%  32\%  34\% &
Tracept  No. changed                                    39    54   119   115 &
          - Event changed                             95\%  81\%  86\%  92\% &
          - Func changed                              46\%  54\%  32\%  51\% &